##  Import Library

In [4]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import datetime
from datetime import datetime
import nltk
from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from pandas_datareader import data as web
import os
import string
import requests
import datetime
import matplotlib.pyplot as plt
import urllib.request
import datetime
from datetime import datetime
import nltk
from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from pandas_datareader import data as web
import os
import string
import requests
import datetime

### Fetching Stock Price From Yahoo Finance

In [5]:
# Stock Price
def stock(stock_type, start, end):
    sp500stock = web.get_data_yahoo([stock_type], start,end)
    return sp500stock
    
stock_type = '^GSPC'
start=datetime.datetime(2017, 1, 1)
end=datetime.datetime(2018,10, 1)

stock_data = stock(stock_type, start, end)

In [6]:
stock_data.head(5)

Attributes,High,Low,Open,Close,Volume,Adj Close
Symbols,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078
2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,2270.750000
2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,2269.000000
2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,2276.979980
2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,2268.899902


#### Due to stock price date is not continuously, so we need to find the date with open markcket

In [7]:
open_date = stock_data.index
len(open_date)

440

### Web Scraping from Wall Street Journal

In [8]:
#Scraping news from the the corresponding date we found
date_loop_list = []
for i in range(len(open_date)):
    d1 = open_date[i]
    date_loop_list.append(d1.date())
print(len(date_loop_list))

440


In [9]:
# Web Scraping from Wall Street Journal
def news(dateloop):
#     date_list = [base - datetime.timedelta(days=x) for x in range(daterange)]
#     date_loop_list = []
#     for i in date_list:
#         dt = datetime.date.strftime(i, '%Y-%m-%d')
#         date_loop_list.append(dt)
    
    All_News=[]
    for date in dateloop:
        url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
        page = requests.get(url)    
        result = BeautifulSoup(page.content,'lxml')
        UL = result.find_all('ul', class_ = 'newsItem')
        body1 = UL[0].find_all('p')
        everyday_results = []
        Headlines_Day = []
        Abstracts_Day = []
        for news_ in range(len(body1)):
            body2 = body1[news_].get_text()
            body3 = body2.strip().split('\n ')
            Headlines_item = body3[0]
            Abstracts_item = body3[-1]
            Headlines_Day.append(Headlines_item)
            Abstracts_Day.append(Abstracts_item)
        everyday_results = [date, Headlines_Day, Abstracts_Day]
        All_News.append(everyday_results)
    return All_News

#### What news data looks like after scraping from website

In [10]:
output_news = news(date_loop_list[0:5])
output_news

[[datetime.date(2017, 1, 3),
  ['Photos of the Day, Asia: Jan. 3 ',
   'Qualcomm Looks Beyond Smartphones With New Chip  ',
   'Heard on the Street ',
   'Trump Knocks U.S. Spy Agencies Over Russia Hacking Review  ',
   'Exxon Mobil Cashes Out Ex-CEO Tillerson Ahead of Confirmation Hearings ',
   'Sullivan Partner in Lead for SEC Chairman ',
   'Business Watch ',
   'Senate Moves to Dismantle Health Law ',
   'Faraday Future Faces Crucial Test With New Electric Car ',
   'Pakistani Activist Faces Blasphemy Charge, Fatwa     ',
   'Trump’s Cabinet Nominees Start Meeting With Senators  ',
   'Israel’s Netanyahu Says Misconduct Allegations Are ‘Persecution’  ',
   'Mexican Officials Caught Off Guard by Ford’s Plant Cancellation  ',
   'Corrections & Amplifications ',
   'Donald Trump to Hold Jan. 11 Press Conference in New York  ',
   'New York Office Market Cooled in Election Year ',
   'China Shrugs Off Trump Twitter Jab on North Korea ',
   'Earnings Recovery Set to Propel Stocks Highe

### Convert the web scraping results to dataframe

In [11]:
def news_to_dataframe(output_news):
    news_df = pd.DataFrame()
    date = []
    news_title = []
    for day in output_news:
        date.append(day[0])  # date
        news_title.append(day[1]) # news_title
    news_df['Date'] = date
    news_df['News_title'] = news_title 
    return news_df


In [12]:
news_to_dataframe(output_news) 

,Date,News_title
0,2017-01-03,"[Photos of the Day, Asia: Jan. 3 , Qualcomm Lo..."
1,2017-01-04,"[Heard on the Street , Audi Pushes Toward Full..."
2,2017-01-05,[Can World’s Miners Dig Up Another Year of Str...
3,2017-01-06,"[Photos: Shooting at Fort Lauderdale Airport ,..."
4,2017-01-09,"[Heard on the Street , Senate Report: Backpage..."
